In [16]:
# set up earth engine
import ee
ee.Initialize()

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [2]:
# import prisons
prison_centroids = ee.FeatureCollection("projects/ee-ccmothes/assets/prison_centroids")


In [3]:
# define date range
startDate = "2012-01-01"
endDate = "2022-12-31"


In [4]:
# Define function to convert Kelvin to Celcius
def toCelciusDay(image):
  lst = image.select('LST_Day_1km').multiply(0.02).subtract(273.15)
  overwrite = True
  result = image.addBands(lst, ['LST_Day_1km'], overwrite)
  return result

In [5]:
# Quality mask; code adopted from https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/
def bitwiseExtract(input, fromBit, toBit):
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return input.rightShift(fromBit).bitwiseAnd(mask)


#Let's extract all pixels from the input image where
#Bits 0-1 <= 1 (LST produced of both good and other quality)
#Bits 2-3 = 0 (Good data quality)
#Bits 4-5 Ignore, any value is ok
#Bits 6-7 = 0 (Average LST error ≤ 1K)
def applyQaMask(image):
  lstDay = image.select('LST_Day_1km')
  qcDay = image.select('QC_Day')
  qaMask = bitwiseExtract(qcDay, 0, 1).lte(1)
  dataQualityMask = bitwiseExtract(qcDay, 2, 3).eq(0)
  lstErrorMask = bitwiseExtract(qcDay, 6, 7).lte(1)
  mask = qaMask.And(dataQualityMask).And(lstErrorMask)
  return lstDay.updateMask(mask)


In [8]:
#import MODIS
modisdata = ee.ImageCollection('MODIS/061/MYD11A1') \
  .filterDate(ee.Date(startDate),ee.Date(endDate)) \
  .filter(ee.Filter.calendarRange(6, 8,'month'))
  

#print(modisdata.first())

In [9]:
# Apply processing functions
lst_day_processed = modisdata.map(toCelciusDay).map(applyQaMask)

In [11]:
# Now calculate average summer day temperature across date range
summer_day_lst = lst_day_processed.select('LST_Day_1km').median()

In [12]:

# Extract values at prison centroids
sampled_points = summer_day_lst.sampleRegions(
  collection=prison_centroids,
  scale=1000,
  geometries=True
)                         
    

In [15]:
print(ee.Feature(None, sampled_points))

ee.Feature({
  "functionInvocationValue": {
    "functionName": "Feature",
    "arguments": {
      "metadata": {
        "functionInvocationValue": {
          "functionName": "Image.sampleRegions",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.loadTable",
                "arguments": {
                  "tableId": {
                    "constantValue": "projects/ee-ccmothes/assets/prison_centroids"
                  }
                }
              }
            },
            "geometries": {
              "constantValue": true
            },
            "image": {
              "functionInvocationValue": {
                "functionName": "reduce.median",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.map",
                      "arguments": {
                        "baseAlg